# Import Libraries

In [ ]:
!pip install -q -U google-cloud-documentai google-cloud-storage Pillow google-cloud-translate google-generativeai
!pip install --upgrade openai
!pip install PyPDF2
!pip install tiktoken
!pip install nltk
!pip install langchain
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00

In [ ]:
import os
import openai
import tiktoken
import requests
import tarfile
import PyPDF2
import pandas as pd
from io import BytesIO
from PyPDF2 import PdfReader
import time
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "herbaria-ai-3c860bcb0f44.json"
# OpenAI API Key
api_path = '/content/drive/MyDrive/Colab Notebooks/openai_apikey.txt'
API_KEY = ""
with open(api_path, 'r') as file:
    API_KEY = file.readline().strip()

Mounted at /content/drive


# Batch Processing

In [ ]:
import pandas as pd
import requests
import json
from google.api_core.client_options import ClientOptions
from google.cloud import documentai_v1 as documentai
from google.cloud.documentai_v1.types import RawDocument
from google.cloud import translate_v2 as translate
import google.generativeai as genai
from google.colab import files
import zipfile
import os
import io

# dataframe setup
# results_df = pd.DataFrame(columns=["Filename", "Extracted Text", "Translated Text"])
results_df = pd.DataFrame(columns=["Filename", "Extracted Text"])


# GCP details
project_id = "herbaria-ai"
location = "us"
processor_id = "4307b078717a399a"

# def translate_text(text, target_language="en"):
#     translate_client = translate.Client()
#     result = translate_client.translate(text, target_language=target_language)
#     return result["translatedText"]

def batch_process_documents(file_path: str, file_mime_type: str) -> tuple:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")
    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    with open(file_path, "rb") as file_stream:
        raw_document = RawDocument(content=file_stream.read(), mime_type=file_mime_type)

    name = client.processor_path(project_id, location, processor_id)
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)
    result = client.process_document(request=request)

    extracted_text = result.document.text
    # translated_text = translate_text(extracted_text)
    # return extracted_text, translated_text
    return extracted_text

def find_images(directory, extensions=('.jpeg', '.jpg', 'png')):
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.lower().endswith(extensions) and not filename.startswith('.'):
                yield os.path.join(root, filename)

def main():
    global results_df
    results_df = results_df.iloc[0:0]  # clear old df

    print("Please upload a zip file containing JPEG/JPG files to process:")
    uploaded_files = files.upload()

    for filename in uploaded_files.keys():
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(io.BytesIO(uploaded_files[filename]), 'r') as zip_ref:
            zip_ref.extractall("extracted_files")

        image_files = list(find_images("extracted_files"))
        print(f"Found {len(image_files)} image files for processing.")

        for file_path in image_files:
            try:
                print(f"Processing {os.path.basename(file_path)}...")
                # extracted_text, translated_text = batch_process_documents(file_path, "image/jpeg")
                extracted_text = batch_process_documents(file_path, "image/jpeg")
                new_row = pd.DataFrame([{
                    "Filename": os.path.basename(file_path),
                    "Extracted Text": extracted_text
                    # "Translated Text": translated_text
                }])
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            except Exception as e:
                print(f"An error occurred while processing {file_path}: {e}")

if __name__ == "__main__":
    main()

Please upload a zip file containing JPEG/JPG files to process:


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
import io
import zipfile
import streamlit as st
from google.cloud import documentai_v1 as documentai
from google.api_core.client_options import ClientOptions
from google.cloud.documentai_v1.types import RawDocument

# Streamlit setup
st.title("Document Processor")

# GCP details
project_id = "herbaria-ai"
location = "us"
processor_id = "4307b078717a399a"

# Dataframe setup
results_df = pd.DataFrame(columns=["Filename", "Extracted Text"])

@st.cache(allow_output_mutation=True)
def batch_process_documents(file_path: str, file_mime_type: str) -> str:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")
    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    with open(file_path, "rb") as file_stream:
        raw_document = RawDocument(content=file_stream.read(), mime_type=file_mime_type)

    name = client.processor_path(project_id, location, processor_id)
    request = documentai.ProcessRequest(name=name, raw_document=raw_document)
    result = client.process_document(request=request)

    return result.document.text

def find_images(directory, extensions=('.jpeg', '.jpg', '.png')):
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.lower().endswith(extensions) and not filename.startswith('.'):
                yield os.path.join(root, filename)

# Streamlit file uploader
uploaded_file = st.file_uploader("Please upload a ZIP file containing image files", type='zip')
if uploaded_file:
    with zipfile.ZipFile(uploaded_file, 'r') as zip_ref:
        zip_ref.extractall("extracted_files")

    image_files = list(find_images("extracted_files"))
    st.write(f"Found {len(image_files)} image files for processing.")

    if st.button('Process Documents'):
        for file_path in image_files:
            try:
                st.write(f"Processing {os.path.basename(file_path)}...")
                extracted_text = batch_process_documents(file_path, "image/jpeg")
                new_row = pd.DataFrame([{
                    "Filename": os.path.basename(file_path),
                    "Extracted Text": extracted_text
                }])
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            except Exception as e:
                st.write(f"An error occurred while processing {file_path}: {e}")

        st.dataframe(results_df)

2024-05-01 14:23:55.367 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
results_df

,Filename,Extracted Text
0,e9466c1f.png,Inches\ncm 1\n2\n་་འཆ༤\nSHARAD\n99\n2\n5\n3\n4...
1,e91a14e7.png,Flora of Pan-Himalaya\n中国科学院植物研究所标本馆(PE)\n采集人:...
2,e9283645.png,植物\n中国科学院植物研究所\n植物标本室\n日期 1987年 10\n採集人\n食 沈保安...
3,e932a60e.png,重庆市药物种植研究所\n植物标本采集记录卡\n采集日期:\n采集人:刘正宇\n2008年7月...
4,e93085fc.png,植物\n中国科学院植物研究所\n植物标本室\n日期 19 2009年9 月21 日\nэтн...
...,...,...
995,e9378cbf.png,湖南植物\n中国科学院植物研究所\n植物标本室。16584\n刘林翰\n1963年11月7日...
996,e92e3337.png,2\n2\nInches\ncm 1\n6\n5\n4\n采集日期\n中国科学院植物研究所\...
997,e9195d4a.png,Flora of Yunnan 云南植物\nFamily: Balsaminaceae\nN...
998,e94aeb44.png,中国科学院植物研究所\n植物标本室\n采集日期\n2008-7-20\n采集人/采集号 巴山...


In [ ]:
result_path = '/content/samples_1000_documentai_results_df.csv'
results_df.to_csv('result_path', encoding='utf-8-sig')

# Gemini Setup

In [ ]:
import pathlib
import textwrap
import json
import pandas as pd
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

results_df = pd.read_csv(result_path)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
genai.configure(api_key='')
# genai.configure(api_key='')

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
# model = genai.GenerativeModel('gemini-1.5-pro-latest')
model = genai.GenerativeModel('gemini-pro')

In [ ]:
results_df

,Unnamed: 0,Filename,Extracted Text
0,0,e9544621.png,植物\n中国科学院植物研究所\n·植物标本室\n日期\n1982 年\n9\n月26 日\n...
1,1,e92cd649.png,cm 1\n4\n5\n6\n1261 14\nE\nInches\n2.\n答曰\n80-...
2,2,e93d793b.png,中国科学院植物研究所\n植物标本室\n采集日期\n2008-7-26\n采集人/采集号 巴山...
3,3,e9423e3c.png,植物\n中国科学院植物研究所\n植物标本室\n日 期 1986年\n6月\n23日\n及數 ...
4,4,e923dcac.png,湖南科技大学生命科学学院植物标本室\nHerbarium\nHunan University...
...,...,...,...
95,95,e9339d52.png,植物\n中国科学院植物研究所\n植物标本室\n日期 19 834\n2\n月\n10\n「採...
96,96,e95c4667.png,cm 1\nInches\nTTT\n2\n3\n4\n5\n6\n2\n277648\n8...
97,97,e93a818d.png,FLORA OF VIET NAM\nCol. L. Q. Li: N. T. Hiep: ...
98,98,e918a8a7.png,广西植物\n科名 凤仙花科\n种名 Balsaminaceaea\n中名 耳叶棒凤仙——\n...


from matplotlib import pyplot as plt
results_df['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Unnamed: 0']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Unnamed: 0'}, axis=1)
              .sort_values('Unnamed: 0', ascending=True))
  xs = counted['Unnamed: 0']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_df.sort_values('Unnamed: 0', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Unnamed: 0')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
results_df['Unnamed: 0'].plot(kind='line', figsize=(8, 4), title='Unnamed: 0')
plt.gca().spines[['top', 'right']].set_visible(False)

# OpenAI Setup

In [ ]:
OpenAI

openai.OpenAI

In [ ]:
from openai import OpenAI

drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "herbaria-ai-3c860bcb0f44.json"
gdrive_path = '/content/drive/MyDrive/Colab Notebooks/DS549/'
gdrive_result_path = '/content/drive/MyDrive/Colab Notebooks/DS549_results/'
# OpenAI API Key
api_path = '/content/drive/MyDrive/Colab Notebooks/openai_apikey.txt'
API_KEY = ""
with open(api_path, 'r') as file:
    API_KEY = file.readline().strip()

client = OpenAI(api_key=API_KEY)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
collector_system_prompt = "\
    You are a Professor of Organismic and Evolutionary Biology of Harvard University. \
    Your are tasked to examine the labeled texts from the result of a OCR scan on a plant specimen. \
    Your task is to find out who is the collector of this specimen. If the collector's name is in another language than English, \
    output the original language AND translate it to English. \
    Output the collector's only. Reasoning is not needed. \
    Follow the examples below. If the answer is not \
    exact, try your best, but do not use excess wording. If you are completely \
    unsure or there is no answer, insert UNKNOWN. \
"
location_system_prompt = "\
    You are a Professor of Organismic and Evolutionary Biology of Harvard University. \
    Your are tasked to examine the labeled texts from the result of a OCR scan on a plant specimen. \
    Your task is to find out where is the location of the collection. If the location's name is in another language than English, \
    output the original language AND translate it to English while keeping only province name or municipality name. \
    Output the location only. Reasoning is not needed. \
    Follow the examples below. If the answer is not \
    exact, try your best, but do not use excess wording. If you are completely \
    unsure or there is no answer, insert UNKNOWN. \
"
taxon_system_prompt = "\
    You are a Professor of Organismic and Evolutionary Biology of Harvard University. \
    Your are tasked to examine the labeled texts from the result of a OCR scan on a plant specimen. \
    Your task is to find out what is the taxon name of this specimen. If the specimen's taxon name is presented in English, output the English scientific name. \
    If there presents both a Chinese name and English scientific name, output the English scientific name. \
    Output specimen's taxon name only. Reasoning is not needed. \
    Follow the examples below. If the answer is not \
    exact, try your best, but do not use excess wording. If you are completely \
    unsure or there is no answer, insert UNKNOWN. \
"
collection_date_system_prompt = "\
    You are a Professor of Organismic and Evolutionary Biology of Harvard University. \
    Your are tasked to examine the labeled texts from the result of a OCR scan on a plant specimen. \
    Your task is to find out what is the collection date of this specimen. Usually there are two dates in the text. \
    One collection date and one identification date. Assume the older date is the collection date. \
    Output the collection date only. Reasoning is not needed. \
    Output the date in YYYYDDMM format. \
    If only the years and months are given, still output the date in YYYYMMDD format, \
    but replace DD with 00, so YYYYMM00. \
    Follow the examples below. If the answer is not \
    exact, try your best, but do not use excess wording. If you are completely \
    unsure or there is no answer, insert UNKNOWN. \
"

In [ ]:
print(taxon_system_prompt)

    You are a Professor of Organismic and Evolutionary Biology of Harvard University.     Your are tasked to examine the labeled texts from the result of a OCR scan on a plant specimen.     Your task is to find out what is the taxon name of this specimen. If the specimen's taxon name is presented in English, output the English scientific name.     If there presents both a Chinese name and English scientific name, output the English scientific name.     Output specimen's taxon name only. Reasoning is not needed.     Follow the examples below. If the answer is not     exact, try your best, but do not use excess wording. If you are completely     unsure or there is no answer, insert UNKNOWN. 


In [ ]:
shots = \
{
    "中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集人/采集 赵常明 EX1376 采集地 湖北省兴山县龙门河自然保护区36拐 生境 阔叶林缘 海拔 1600ml 性状 草本 高度 0.3mm 花果中学 绿 中文名 珍珠菜 学名 科名 报春花科 鉴定人 鉴定日期 备注 CHINESE NATIONAL HERBARIUM (PE) 01868342 No 2009128 中国科学院 植物研究所 标本馆 PE Lysimachia fortunei Maxim. DET. Gay Has 2011.3.24 Inches cm 1 3 9 2 5 湖北省 HUPEH 216":
     {"Collector":"Changming Zhao 赵常明",
      "Location":"Hubei Province 湖北省兴山县龙门河自然保护区36拐",
      "Taxon":"Lysimachia fortunei Maxim.",
      "Collection Date":"20070923"
     },
    "Flora of Hubei Family Name Lygodiaceae Lygodium japonicum Locality 湖北省:宜昌市,大老岭 Habitat 林下 Alt. 400 m Notes Lon./Lat Coll. 张宪春、武 No. 5373 Date 2008-11-23 Dupl. 2 Inches cm 1 2 w 4 5 巡 N° 2329164 中国科学院 植物研究所 标本馆 PE 湖北省 HUPEH CHINESE NATIONAL HERBARIUM PE Lygodium japonicum (Thunb.) Sw. 5373 X. C. Zhang 2008-11-30 01870170":
    {"Collector":"Xianchun Zhang 张宪春",
      "Location":"Hubei Province 湖北省:宜昌市,大老岭",
      "Taxon":"Lygodium japonicum (Thunb.) Sw.",
      "Collection Date":"20081123"
     },
    "【采集人及采集号:海峡两岸植物采集队 09007 采集日期:20090802 采集地点:湖南省通道县万佛山风景区 生境:丹霞地貌 湖南省 HUNAN 海拔: 300-500 形态:蕨类 高度:m 花特征: 果实特征: 其他特征: 科名: 方言名:江南卷柏 学名: Selaginella moellendorffii NO 2304233) 中国科学院 植物研究所 标本馆 PE CHINESE NATIONAL HERBARIUM (PE) 01871034 газоват |cm 11 2 3 Inches 4 5 6 2. PE 1928 No.09007 Selaginella moellendorffii Hieron. DET. Li Zhongyang 2013-06-18":
    {"Collector":"Cross-Strait Plant Collecting Team 海峡两岸植物采集队",
      "Location":"Hunan Province 湖南省通道县万佛山风景区",
      "Taxon":"Selaginella moellendorffii Hieron.",
      "Collection Date":"20090802"
     },
    "四川大学生物系四川植物采集記錄 标本室号数: 宋集人:周洪富粟和毅采集号数 1084.19 采集日期: 195644 6月 產地,环境,地形: 四川奉节兴隆区茅草具 瓦刚塞 海拔 1800 * 生态: 路旁林边 性狀:乔木、灌木、草本或藤本 基暗褐色具黃色 米、胸高直徑腺点,幼枝绿色, 高度 叶柄黄绿色光滑,表面黄绿色, 背面淡黄绿色,两面均光滑, 北梗绿色光滑,花蔓绿色花瓣淡 黄绿色,雄蕊5枚,花丝类绿色花 果 药淡黄色, 附記: 常见 科名: Celastracea俗名: 学名: Celastrus 0909203 中國科學院 植物研究所 標本室, 四川省 SZECHUAN 昇汞HgCl2 灰叶南蛇藤 Celastrus glaucophyllus Rehd. et Wils. Det. 沐先运 (Xian-Yun Mu) 2011-3-30 四 川 植 物 4 CHINESE NATIONAL ERBARIUM (PE) 01881178 采集地点:四川东部:奉节县、巫山县 采集人:四川大学川东植物调查队 鑑定人: 时期 号数 年 月 日 四川大学生物系植物标本室":
    {"Collector":"Hongfu Zhou and Yi Su 周洪富、粟和毅",
      "Location":"Sichuan Province 四川奉节兴隆区茅草具",
      "Taxon":"Lysimachia fortunei Maxim",
      "Collection Date":"20070923"
     },
    "中国科学院植物研究所 植物标本室 采集日期 2008-7-9 采集人/采集号 巴山采集队 0216 采集地 重庆市城口县鲜花村云盘塞 东经: 109°05'144\" 北纬: 31° 47' 155\" 生境 山坡路旁 海拔 1420mm 性狀 草本 高度 0.1-0.15m 果 中文名 石书 学名 鉴定人 鉴定日期 备注 3份 CHINESE NATIONAL HERBARIUM PE 01851414 No 2330236 SZECHUAN cm 1 2 3 4 5 6 7 中国科学院 植物研究所 标本馆 PE Pyrrosia lingua (Thunb.) Farwell (0216) 何丽娟 2008-12-1":
    {"Collector":"Bashan Collecting Team 巴山采集队",
      "Location":"Chongqing City 重庆市城口县鲜花村云盘塞",
      "Taxon":"Pyrrosia lingua",
      "Collection Date":"20080709"
     }
}

In [ ]:
# 5 Shots is too expensive
# base_prompt = \
# """
# Input 0:
# {shot0_input}

# Output 0:
# {shot0_output}

# Input 1:
# {shot1_input}

# Output 1:
# {shot1_output}

# Input 2:
# {shot2_input}

# Output 2:
# {shot2_output}

# Input 3:
# {shot3_input}

# Output 3:
# {shot3_output}

# Input 4:
# {shot4_input}

# Output 4:
# {shot4_output}

# Input:
# {input_text}

# Output:
# """

# 3 Shots
base_prompt = \
"""
Input 0:
{shot0_input}

Output 0:
{shot0_output}

Input 2:
{shot2_input}

Output 2:
{shot2_output}

Input 4:
{shot4_input}

Output 4:
{shot4_output}

Input:
{input_text}

Output:
"""


collector_prompt = base_prompt.format(
    shot0_input = list(shots.keys())[0],
    shot0_output = list(shots.values())[0]['Collector'],
    # shot1_input = list(shots.keys())[1],
    # shot1_output = list(shots.values())[1]['Collector'],
    shot2_input = list(shots.keys())[2],
    shot2_output = list(shots.values())[2]['Collector'],
    # shot3_input = list(shots.keys())[3],
    # shot3_output = list(shots.values())[3]['Collector'],
    shot4_input = list(shots.keys())[4],
    shot4_output = list(shots.values())[4]['Collector'],
    input_text = '{input_text}'
)

location_prompt = base_prompt.format(
    shot0_input = list(shots.keys())[0],
    shot0_output = list(shots.values())[0]['Location'],
    # shot1_input = list(shots.keys())[1],
    # shot1_output = list(shots.values())[1]['Location'],
    shot2_input = list(shots.keys())[2],
    shot2_output = list(shots.values())[2]['Location'],
    # shot3_input = list(shots.keys())[3],
    # shot3_output = list(shots.values())[3]['Location'],
    shot4_input = list(shots.keys())[4],
    shot4_output = list(shots.values())[4]['Location'],
    input_text = '{input_text}'
)

taxon_prompt = base_prompt.format(
    shot0_input = list(shots.keys())[0],
    shot0_output = list(shots.values())[0]['Taxon'],
    # shot1_input = list(shots.keys())[1],
    # shot1_output = list(shots.values())[1]['Taxon'],
    shot2_input = list(shots.keys())[2],
    shot2_output = list(shots.values())[2]['Taxon'],
    # shot3_input = list(shots.keys())[3],
    # shot3_output = list(shots.values())[3]['Taxon'],
    shot4_input = list(shots.keys())[4],
    shot4_output = list(shots.values())[4]['Taxon'],
    input_text = '{input_text}'
)

collection_date_prompt = base_prompt.format(
    shot0_input = list(shots.keys())[0],
    shot0_output = list(shots.values())[0]['Collection Date'],
    # shot1_input = list(shots.keys())[1],
    # shot1_output = list(shots.values())[1]['Collection Date'],
    shot2_input = list(shots.keys())[2],
    shot2_output = list(shots.values())[2]['Collection Date'],
    # shot3_input = list(shots.keys())[3],
    # shot3_output = list(shots.values())[3]['Collection Date'],
    shot4_input = list(shots.keys())[4],
    shot4_output = list(shots.values())[4]['Collection Date'],
    input_text = '{input_text}'
)


In [ ]:
print(taxon_prompt)


Input 0:
中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集人/采集 赵常明 EX1376 采集地 湖北省兴山县龙门河自然保护区36拐 生境 阔叶林缘 海拔 1600ml 性状 草本 高度 0.3mm 花果中学 绿 中文名 珍珠菜 学名 科名 报春花科 鉴定人 鉴定日期 备注 CHINESE NATIONAL HERBARIUM (PE) 01868342 No 2009128 中国科学院 植物研究所 标本馆 PE Lysimachia fortunei Maxim. DET. Gay Has 2011.3.24 Inches cm 1 3 9 2 5 湖北省 HUPEH 216

Output 0:
Lysimachia fortunei Maxim.

Input 2:
【采集人及采集号:海峡两岸植物采集队 09007 采集日期:20090802 采集地点:湖南省通道县万佛山风景区 生境:丹霞地貌 湖南省 HUNAN 海拔: 300-500 形态:蕨类 高度:m 花特征: 果实特征: 其他特征: 科名: 方言名:江南卷柏 学名: Selaginella moellendorffii NO 2304233) 中国科学院 植物研究所 标本馆 PE CHINESE NATIONAL HERBARIUM (PE) 01871034 газоват |cm 11 2 3 Inches 4 5 6 2. PE 1928 No.09007 Selaginella moellendorffii Hieron. DET. Li Zhongyang 2013-06-18

Output 2:
Selaginella moellendorffii Hieron.

Input 4:
中国科学院植物研究所 植物标本室 采集日期 2008-7-9 采集人/采集号 巴山采集队 0216 采集地 重庆市城口县鲜花村云盘塞 东经: 109°05'144" 北纬: 31° 47' 155" 生境 山坡路旁 海拔 1420mm 性狀 草本 高度 0.1-0.15m 果 中文名 石书 学名 鉴定人 鉴定日期 备注 3份 CHINESE NATIONAL HERBARIUM PE 01851414 No 2330236 SZECHUAN cm 1 2 3 4 5 6 7 中

In [ ]:
attributes = ['collector', 'location', 'taxon', 'collection_date']
df_attributes = pd.DataFrame(attributes, columns=['Attribute'])
combined_system_prompt = [collector_system_prompt, location_system_prompt, taxon_system_prompt, collection_date_system_prompt]
df_system_prompts = pd.DataFrame(combined_system_prompt, columns=['System Prompt'])
combined_user_prompt = [collector_prompt, location_prompt, taxon_prompt, collection_date_prompt]
df_user_prompts = pd.DataFrame(combined_user_prompt, columns=['User Prompt'])
df_prompts = pd.concat([df_attributes, df_system_prompts, df_user_prompts], axis=1)

df_prompts.head()

,Attribute,System Prompt,User Prompt
0,collector,You are a Professor of Organismic and Evol...,\nInput 0:\n中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集...
1,location,You are a Professor of Organismic and Evol...,\nInput 0:\n中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集...
2,taxon,You are a Professor of Organismic and Evol...,\nInput 0:\n中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集...
3,collection_date,You are a Professor of Organismic and Evol...,\nInput 0:\n中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集...


In [ ]:
for index, row in df_prompts.iterrows():
    print(row['Attribute'])
    print(len(row['System Prompt']))
    print(len(row['User Prompt']))

collector
616
1144
location
668
1140
taxon
698
1129
collection_date
811
1078


In [ ]:
def get_prompt_response(system_prompt, user_prompt, input_text):
    """ Calls and performs prompt response on prompt.
        Returns response from ChatCompletion.
    """

    openai.api_key = API_KEY
    # MODEL = "gpt-3.5-turbo-0125"
    MODEL = "gpt-4-turbo-2024-04-09"
    # Your response should contain only JSON. If you \
    # encounter two dates in an input, prefer the earliest. If the answer is not \
    # exact, try your best, but do not use excess wording. If you are completely \
    # unsure or there is no answer, insert UNKNOWN.
    SYSTEM_PROMPT = system_prompt
    USER_PROMPT = user_prompt.format(input_text=input_text)
    TEMPERATURE_LIMIT = 0
    TOKEN_LIMIT = 1024

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            # First insert system prompt
            {"role": "system", "content": SYSTEM_PROMPT},
            # Second review the paper
            {"role": "user", "content": USER_PROMPT},
            # Third answer the NIPS review questions
            {"role": "user", "content": "{}".format(prompt)},
            ],
        temperature=TEMPERATURE_LIMIT,
        max_tokens=TOKEN_LIMIT,
    )

    api_response_content = response.choices[0].message.content
    # api_response_content = response['choices'][0]['message']['content']

    return api_response_content


    # try:

    # except openai.error.InvalidRequestError as e:
    #     print("Invalid Request:\n{}".format(e))
    #     return

    # except openai.error.RateLimitError as e:
    #     print("Rate limit reached, pausing for a while. Error Message:\n{}".format(e))
    #     time.sleep(15)  # pause for 15 seconds

In [ ]:
df_prompts.iloc[0:1]

,Attribute,System Prompt,User Prompt
0,collector,You are a Professor of Organismic and Evol...,\nInput 0:\n中国科学院植物研究所 植物标本室 采集日期 2007-9-23 采集...


In [ ]:
results_df.head()

,Filename,Extracted Text
0,e9466c1f.png,Inches\ncm 1\n2\n་་འཆ༤\nSHARAD\n99\n2\n5\n3\n4...
1,e91a14e7.png,Flora of Pan-Himalaya\n中国科学院植物研究所标本馆(PE)\n采集人:...
2,e9283645.png,植物\n中国科学院植物研究所\n植物标本室\n日期 1987年 10\n採集人\n食 沈保安...
3,e932a60e.png,重庆市药物种植研究所\n植物标本采集记录卡\n采集日期:\n采集人:刘正宇\n2008年7月...
4,e93085fc.png,植物\n中国科学院植物研究所\n植物标本室\n日期 19 2009年9 月21 日\nэтн...


In [ ]:
df_prompts['System Prompt'].iloc[0:1]

0        You are a Professor of Organismic and Evol...
Name: System Prompt, dtype: object

In [ ]:
for p_index, p_row in df_prompts.iterrows():
    print(type(p_row['System Prompt']))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
for index, row in results_df[0:2].iterrows():
    print(row['Filename'].split('.')[0])

e9466c1f
e91a14e7


In [ ]:
# Looping over the csv file
# OpenAI does offer JSON mode, but I do not have time to test it

df_return = pd.DataFrame()

import time
import datetime

begin_time = datetime.datetime.now()

# iterate through all 25 papers
# API failures due to server
# todo: write reliant online logic
for index, row in results_df.iterrows():
# for index, row in df_review_tex.iloc[0:2,].iterrows():

    results = []

    # debug
    start_time = datetime.datetime.now()
    print(row['Filename'])
    print("-------------------------------------")

    # input text = extracted text
    filename = row['Filename']
    input_text = row['Extracted Text']

    # debug
    print(filename)
    elapsed_time = datetime.datetime.now() - start_time
    minutes, seconds = divmod(elapsed_time.seconds, 60)
    # print(f"Elapsed time: {minutes:02d} minutes {seconds:02d} seconds")

    # iterate through all prompts
    for p_index, p_row in df_prompts.iterrows():
    # for p in df_prompts['Prompts']:
        retries = 0

        # debug
        # print("    ", p_row['System Prompt'])
        # print("    ", p_row['User Prompt'])
        print("    -------------------------------------")

        while retries < 2:
            try:
                api_response_content = get_prompt_response(p_row['System Prompt'], p_row['User Prompt'], input_text)
                results.append({
                    'Filename': filename,
                    'Extracted Text': input_text,
                    'Attribute': p_row['Attribute'],
                    'Response': api_response_content
                })
                break

            # Retry 3 times if running into errors
            except openai.error.InvalidRequestError as e:
                print(f"Invalid Requests Error, pausing for a while. Error Message:\n{e}")
                time.sleep(60)  # pause for 60 seconds

            except openai.error.RateLimitError as e:
                print(f"Rate limit reached, pausing for a while. Error Message:\n{e}")
                time.sleep(60)  # pause for 60 seconds

            retries += 1

        # debug
        # print("Prompt completed")
        elapsed_time = datetime.datetime.now() - start_time
        minutes, seconds = divmod(elapsed_time.seconds, 60)
        # print(f"Elapsed time: {minutes:02d} minutes {seconds:02d} seconds")

    df_results = pd.DataFrame(results)
    basefilename = row['Filename'].split('.')[0]
    path = gdrive_path + 'ocr_results_' + basefilename +'.csv'
    print(path)
    df_results.to_csv(path, index=True, encoding='utf-8-sig')
    df_return = pd.concat([df_return, df_results])

end_time = datetime.datetime.now() - start_time
print(f"End time: {minutes:02d} minutes {seconds:02d} seconds")

Streaming output truncated to the last 5000 lines.
-------------------------------------
e921c36b.png
    -------------------------------------
    -------------------------------------
    -------------------------------------
    -------------------------------------
/content/drive/MyDrive/Colab Notebooks/DS549/ocr_results_e921c36b.csv
e91be581.png
-------------------------------------
e91be581.png
    -------------------------------------
    -------------------------------------
    -------------------------------------
    -------------------------------------
/content/drive/MyDrive/Colab Notebooks/DS549/ocr_results_e91be581.csv
e933f702.png
-------------------------------------
e933f702.png
    -------------------------------------
    -------------------------------------
    -------------------------------------
    -------------------------------------
/content/drive/MyDrive/Colab Notebooks/DS549/ocr_results_e933f702.csv
e91879fb.png
-------------------------------------
e9187

In [ ]:
df_return.head()

,Filename,Extracted Text,Attribute,Response
0,e9544621.png,植物\n中国科学院植物研究所\n·植物标本室\n日期\n1982 年\n9\n月26 日\n...,collector,Bosheng Li and Shuzhi Cheng 李渤生、程树志
1,e9544621.png,植物\n中国科学院植物研究所\n·植物标本室\n日期\n1982 年\n9\n月26 日\n...,location,Tibet 西藏省墨脱县校区兴凯一江别塘
2,e9544621.png,植物\n中国科学院植物研究所\n·植物标本室\n日期\n1982 年\n9\n月26 日\n...,taxon,Pilea insolens Wedd.
3,e9544621.png,植物\n中国科学院植物研究所\n·植物标本室\n日期\n1982 年\n9\n月26 日\n...,collection_date,19820926
0,e92cd649.png,cm 1\n4\n5\n6\n1261 14\nE\nInches\n2.\n答曰\n80-...,collector,Yan Luo and Yousheng Chen 罗艳、陈又生


In [ ]:
print("We have a count of {} samples being processed.".format(df_return['Filename'].nunique()))
print("Each with a count of {} prompts being processed.".format(len(df_return)/df_return['Filename'].nunique()))

We have a count of 100 samples being processed.
Each with a count of 4.0 prompts being processed.


In [ ]:
# 1. Pivot the DataFrame
df_pivot = df_return.pivot(index=['Filename', 'Extracted Text'], columns='Attribute', values='Response')

# # 2. Reorder the columns
# desired_order = df_prompt['Prompts'].tolist()
# desired_index_order = df_review_tex['Title'].tolist()

# df_pivot = df_pivot.reindex(desired_order, axis=1)
# df_pivot = df_pivot.reindex(desired_index_order, level='Title')
df_pivot

,Attribute,collection_date,collector,location,taxon
Filename,Extracted Text,,,,
47a03bef.png,"采集日期\n中国科学院植物研究所\n植物标本室\n采集人/采集号 巴山采集队\n2008-7-26\n2698\n采集地 重庆市开县百泉乡上华村\n东经:\n108°44795""\n北纬: 31°38' 354""\n生境\n海拔\n高度\n花\n灌林中\n1763 m\n性状\n果\n中文名\n科名 Pittosporaceae\n学名\n鉴定人\n鉴定日期\n备注\n2份\nCHINESE NATIONAL HERBARIUM (PE)\n01868300\nN° 2264577\n*\n中国科学院\n植物研究所\n标本馆\nPE\nInches\n6\n2\n5\nพว\n2\n4\nPittosporum glabratum Lindl. var. neriifolium Rehd. & Wils.\n林祁\n2011-04-11\n",20080726,Bashan Collecting Team 巴山采集队,Chongqing City 重庆市开县百泉乡上华村,Pittosporum glabratum Lindl. var. neriifolium ...
e9133452.png,"四川大学生物系四川植物采集記錄\n标本室号数:\n宋集人:周洪富粟和毅采集号数 1084.19\n采集日期:\n195644 6月\n產地,环境,地形: 四川奉节兴隆区茅草具\n瓦刚塞\n海拔\n1800\n*\n生态: 路旁林边\n性狀:乔木、灌木、草本或藤本 基暗褐色具黃色\n米、胸高直徑腺点,幼枝绿色,\n高度\n叶柄黄绿色光滑,表面黄绿色,\n背面淡黄绿色,两面均光滑,\n北梗绿色光滑,花蔓绿色花瓣淡\n黄绿色,雄蕊5枚,花丝类绿色花\n果\n药淡黄色,\n附記: 常见\n科名: Celastracea俗名:\n学名: Celastrus\n0909203\n中國科學院\n植物研究所\n標本室,\n四川省\nSZECHUAN\n昇汞HgCl2\n灰叶南蛇藤 Celastrus glaucophyllus Rehd. et Wils.\nDet. 沐先运\n(Xian-Yun Mu) 2011-3-30\n四 川 植 物\n4\nCHINESE NATIONAL ERBARIUM (PE)\n01881178\n采集地点:四川东部:奉节县、巫山县\n采集人:四川大学川东植物调查队\n鑑定人:\n时期\n号数\n年 月 日\n四川大学生物系植物标本室\n",195606,Sichuan University Eastern Sichuan Plant Surve...,Sichuan Province 四川奉节兴隆区茅草具,Lysimachia fortunei Maxim.
e917f44a.png,"NO 2258774\n中国科学院\n植物研究所\n标本馆\nPE\nCHINESE NATIONAL HERBARIUM (PE)\n01878935\nInches\nJom\n3\n4\n61\n1931\n贵州省\nKWEICHOW\nImpatias quizhonents V.4. Chen\n陈艺林\n2012年3月12 日\nPLANTS OF GUIZHOU, CHINA\nBalsaminaceae\nImpatiens pritzelii Hook. f.\nDet.: Y. L. Chen 1987\nSongtao Xian, vicinity of Lengjiaba in the vicinity of\nthe confluence of the Xiaohe and Dahe rivers, NE side\nof the Fanjing Shan mountain range.\nElevation 820-1120 m\nMoist rock crevices in semi-shade, flowers whitish\nbecoming pink toward the mouth and on the spur.\nSino-American Guizhou Botanical Expedition no. 1904\n5-9 October 1986\nParticipants: B. Bartholomew, D. E. Boufford, Q. H. Chen\nS.Z. Fang, J. G. Qi, S. A. Spongberg, Z. H. Tsi, Y. L. Tu\nP. S. Wang, Y. H. Xiang and T. S. Ying\n",19861005,Sino-American Guizhou Botanical Expedition 中美贵...,Guizhou Province 贵州省松桃苗族自治县冷家坝附近小河和大河交汇处东北方向梵净山山脉,Impatiens pritzelii Hook. f.
e918a8a7.png,"广西植物\n科名 凤仙花科\n种名 Balsaminaceaea\n中名 耳叶棒凤仙——\n产地 田林岑王老山龙坪至猴子\n经度 E106°23'09.5""\n纬度N24°24'04.7""\n生境山谷,溪边或山坡路边\n海拔 1263\n习性 草本,\n性状花黄色,基部裂片很少,上部\n采集日 10/13/2005\n采集人号于胜祥 No.3733\n份数\n中科院植物所标本馆(PE)\nNo 2094108\n中国科学院\n植物研究所\nCHINESE NATIONAL HERBARIUM (PE)\n标本馆\n01878600\nPE\nInches\nPE 1928\njcm 1\n5\n61\n廣西\nKWANGST\nImpatiens Godinier Book.f.\n陈艺林 2072年2月8日\n",20051013,Shengxiang Yu 于胜祥,Guangxi Province 广西田林岑王老山龙坪至猴子,Impatiens godseffiana Hook.f.
e919e675.png,"重庆市药物种植研究所\n植物标本采集记录卡\n采集日期: 2007\n年4月30日\n采集人:刘正宇\n采集号:180992\n产地: 重庆市 省(市) 3贡县(区、市)\n梓里\n海拔:\n280m\n高度: 2.5-3.2m\n环境: 溪边针叶林F\n性状: 灌木\n根:\n茎(皮):直立、黄绿色,多分枝\n叶单十.草质·绿色\n花: 花黄白色,花蕾绿色\n果及种子。\n用途:\n植物名:\n学名:\n别名:油菜\n科名:\n山茶科\n鉴定人\n附记:\nNo 2021324\n中国科学院\n植物研究所\n标本馆\nPE\nCHINESE NATIONAL HERBARIUM (PE)\n01878017\nTinches\n采期07年02\ncm 1\n4\n51\n重庆药物研究所\n采集人 刘正宇\n采集\n号 180'\nPE\n1928\nSZECHUAN\nCamellia oleifera Abel\nDet. S. X. Yang 2011. 3. 24\n",20070430,Zhengyu Liu 刘正宇,Chongqing City 重庆市省(市) 3贡县(区、市)梓里,Camellia oleifera Abel
...,...,...,...,...,...
e9697263.png,"四川大学生物系四川植物采集記錄,\n本室号数:\n全集人: 天倫 采集号数10)今66\n采集日期: 1952年 9月14日\n產地,环境,地形:碱口县白翅山高家坡\n四川省\nSZECHUAN\n昇汞HgCl2\n海拔1650\n米\n生态:生於灌木林中\n性狀:乔木、灌木、草本或藤本\n高度\n米胸高直徑\n米\n腹平常也缘具前\n办个片雄忝紧花柱一条子房下位,\n果\n附記:少见\n科名:\n学名:\ncm 1 2\n25\n6\n0914678\n俗名:\n中國科學院\n植物研究所\n標本室\nCHINESE NATIONAL HERBARIUM PE\n01836841\n100\nEupatoriceus japonicum Theul.\nDet. 陈艺林\n2009年2月23日\n",19520914,UNKNOWN,Sichuan Province 四川碱口县白翅山高家坡,Eupatoriceus japonicum Theul.
e96a2b5f.png,"植物\n中国科学院植物研究所\n植物标本室\n日期 1986年8月2 日\n柔集人, 金佛山考察队\n1951\n产地: 四川省 南川 金佛山扇子坪去\n环境后轉運中荒地上\n海拔 1450 m 性状\ncm 体高\n木\n草木\nm\n叶对生,柔之分特殊气味\n花浅粉色\n果实\n名泽兰 科名\n菊科\n學名\n附记\nInches\n25\ncm 1 2 3 4 5 6\nNo 1374100\n「中國科學院\n植物研究所\n标本馆\n01836499\n1951\n四川省\nSZECHUAN\nEupatorizem japonicum Thenk-\nDet. 陈艺林\n2009年3月2日\n",19860802,Jinfo Mountain Expedi

In [ ]:
df_pivot.to_csv(gdrive_result_path + '/ocr_results_batch_900.csv', index=True, encoding='utf-8-sig')

In [ ]:
# Function to count 'unknown' variants in each column
def count_unknowns(df):
    # Case insensitive matching
    unknowns_count = df.apply(lambda col: col.str.contains('unknown', case=False, na=False).sum())
    return unknowns_count

# Assuming df_pivot is your DataFrame
unknown_counts = count_unknowns(df_pivot)

# Print the counts for each column
print(unknown_counts)

Attribute
collection_date     6
collector          12
location            7
taxon               2
dtype: int64


In [ ]:
responses = []
counter = 0
# for input_text in results_df["Translated Text"]:
# for input_text in results_df["Extracted Text"]:
for input_text in results_df["Extracted Text"]:
    # need to add error handling
    counter += 1

    prompt = \
"""
Follow the examples below. Your response should contain only JSON. If you
encounter two dates in an input, prefer the earliest. If the answer is not
exact, try your best, but do not use excess wording. If you are completely
unsure or there is no answer, insert UNKNOWN.

Input 1:
{shot1_input}

Output 1:
{{"Collector":"{shot1_output_collector}","Location":"{shot1_output_location}","Taxon":"{shot1_output_taxon}","Date":"{shot1_output_date}"}}

Input 2:
{shot2_input}

Output 2:
{{"Collector":"{shot2_output_collector}","Location":"{shot2_output_location}","Taxon":"{shot2_output_taxon}","Date":"{shot2_output_date}"}}

Input 3:
{shot3_input}

Output 3:
{{"Collector":"{shot3_output_collector}","Location":"{shot3_output_location}","Taxon":"{shot3_output_taxon}","Date":"{shot3_output_date}"}}

Input 4:
{input_text}
Output 4:
"""

    random_pairs = get_random_pairs_list(shots)

    prompt = prompt.format(
    shot1_input = random_pairs[0][0],
    shot1_output_collector = random_pairs[0][1]['Collector'],
    shot1_output_location = random_pairs[0][1]['Location'],
    shot1_output_taxon = random_pairs[0][1]['Taxon'],
    shot1_output_date = random_pairs[0][1]['Date'],

    shot2_input = random_pairs[1][0],
    shot2_output_collector = random_pairs[1][1]['Collector'],
    shot2_output_location = random_pairs[1][1]['Location'],
    shot2_output_taxon = random_pairs[1][1]['Taxon'],
    shot2_output_date = random_pairs[1][1]['Date'],

    shot3_input = random_pairs[2][0],
    shot3_output_collector = random_pairs[2][1]['Collector'],
    shot3_output_location = random_pairs[2][1]['Location'],
    shot3_output_taxon = random_pairs[2][1]['Taxon'],
    shot3_output_date = random_pairs[2][1]['Date'],

    input_text = input_text
    )

    response = model.generate_content(prompt)
    responses.append(response)

    print('Recieved response', counter)

    # limit 10 prompts for now, add handling later
    if counter >= 100:
      break

Recieved response 1
Recieved response 2
Recieved response 3
Recieved response 4
Recieved response 5
Recieved response 6
Recieved response 7
Recieved response 8
Recieved response 9
Recieved response 10
